In [20]:
import numpy as np

from src.utils.utils import get_logger
import logging

from pyspark.sql import SparkSession
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd

from xgboost import XGBClassifier

logger = logging.getLogger(__name__)
logger = get_logger(logger=logger)

In [21]:
spark = SparkSession.builder.master("local[*]").appName("trend_line").getOrCreate()

In [22]:
logger.info("Leitura dos dados")
df_raw = pd.read_parquet(f'../data/interim/data_ready_to_train.parquet')
df_raw["ano"] = df_raw["ano"].astype(int)
df_raw["semestre"] = df_raw["semestre"].astype(int)

2024-03-20 12:13:06,890 - __main__ - INFO - Leitura dos dados
2024-03-20 12:13:06,890 - __main__ - INFO - Leitura dos dados


In [23]:
df_raw = df_raw[df_raw["ano_semestre"] == "202302"].copy() 

In [24]:
# Split train and test
X = df_raw[['ncm_le', 'id_pais_origem_le', 'importador_municipio_le','urf_le', 'ano','semestre']]
y = df_raw["status_le"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

xgbc = XGBClassifier(enable_categorical=True)

xgbc.fit(X_train, y_train)

y_hat = xgbc.predict(X_test)
y_hat_prob = xgbc.predict_proba(X_test)

df_compare = X_test.copy()
df_compare["true_status"] = y_test
df_compare["predicted_status"] = y_hat
df_compare[["prob_0", "prob_1", "prob_2"]] = y_hat_prob

In [25]:
y_hat_prob

array([[0.5228953 , 0.03895073, 0.438154  ],
       [0.2439354 , 0.462324  , 0.29374057],
       [0.44470924, 0.07517921, 0.4801116 ],
       ...,
       [0.24112904, 0.48335436, 0.27551663],
       [0.40559763, 0.02879944, 0.5656029 ],
       [0.49470523, 0.06569737, 0.4395974 ]], dtype=float32)

In [26]:
df_compare["correct"] = df_compare.apply(lambda x: 1 if x["true_status"] == x["predicted_status"] else 0, axis=1)
df_compare["correct"].value_counts()/df_compare.shape[0]

1    0.508967
0    0.491033
Name: correct, dtype: float64

In [ ]:
df_compare[]

In [27]:
# Remoção de outliers agrupado
# Separação de treino e teste

# Deleção dos dados nulos